In [1]:
import os


In [2]:
%pwd

'C:\\Users\\afeef\\OneDrive\\Documents\\PYTHON\\InfoCondense\\InfoCondenseTool\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\afeef\\OneDrive\\Documents\\PYTHON\\InfoCondense\\InfoCondenseTool'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from src.InfoCondense.constants import *
from src.InfoCondense.utils.common import read_yaml, create_directories

In [7]:
class ConfigManager:
    def __init__(
            self, 
            config_path: CONFIG_FILE_PATH,
            params_path: PARAMS_FILE_PATH):
            self.config = read_yaml(config_path)
            self.params = read_yaml(params_path) 
            create_directories([self.config.artifacts_root])
            
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir, 
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        
        return data_transformation_config
    

In [8]:
from transformers import AutoTokenizer
import os
from src.InfoCondense.logging import logger
from datasets import load_dataset, load_from_disk

C:\Python312new\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-07-11 00:26:56,426: INFO: config]: PyTorch version 2.3.1 available.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config =config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length= 1024, truncation=True, padding= True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation= True, padding = True)
            
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        try:
            dataset_samsum = load_from_disk(str(self.config.data_path))
            dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
            target_dir=os.path.join(self.config.root_dir,"samsum_dataset_transformed")
            dataset_samsum_pt.save_to_disk(target_dir)
        except Exception as e:
            raise e
                                       
      
    

In [10]:
try:
    config = ConfigManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config= data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

Trying to open the yaml file at: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\config\config.yaml
[2024-07-11 00:26:57,610: INFO: common]: yaml file: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\config\config.yaml loaded successfully]
Trying to open the yaml file at: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\params.yaml
[2024-07-11 00:26:57,624: INFO: common]: yaml file: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\params.yaml loaded successfully]
[2024-07-11 00:26:57,627: INFO: common]: created directory at: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\artifacts]
[2024-07-11 00:26:57,630: INFO: common]: created directory at: C:\Users\afeef\OneDrive\Documents\PYTHON\InfoCondense\InfoCondenseTool\artifacts\data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]C:\Python312new\Lib\site-packages\transformers\tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 61643.26 examples/s]
